Etape 1 : Installation des bibliothèques.

In [ ]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s

In [ ]:
pip install ffmpeg-python

In [ ]:
pip install pydub librosa scipy

In [ ]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Etape 2 : Chargement de dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Audio/audio_test_file.csv')

In [ ]:
df.head()

,filename,text,full_path
0,cv-other-test/sample-000000.mp3,you killed all three of them for nothing,/content/drive/MyDrive/Audio/cv-other-test/sam...
1,cv-other-test/sample-000001.mp3,will it be warmer in montana,/content/drive/MyDrive/Audio/cv-other-test/sam...
2,cv-other-test/sample-000002.mp3,science has been arguing about the zoological ...,/content/drive/MyDrive/Audio/cv-other-test/sam...
3,cv-other-test/sample-000003.mp3,he wasn't asking for help,/content/drive/MyDrive/Audio/cv-other-test/sam...
4,cv-other-test/sample-000004.mp3,but he granted the travelers three days,/content/drive/MyDrive/Audio/cv-other-test/sam...


Etape 3 : L'application des techniques de Preprocessing sur les audios

In [ ]:
import os
import librosa
import soundfile as sf
from pydub import AudioSegment
from scipy.signal import butter, lfilter

In [ ]:
from scipy.signal import butter, lfilter

def highpass_filter(data, sr, cutoff=100, order=5):
    """
    Applique un filtre passe-haut à un signal audio.

    Paramètres :
    - data : le signal audio (tableau de données)
    - sr : fréquence d'échantillonnage (sample rate)
    - cutoff : fréquence de coupure du filtre (par défaut 100 Hz)
    - order : ordre du filtre (plus l'ordre est élevé, plus la transition est abrupte)

    Retour :
    - Signal filtré (passe-haut)
    """
    # Calcul de la fréquence de Nyquist (moitié de la fréquence d'échantillonnage)
    nyquist = 0.5 * sr

    # Normalisation de la fréquence de coupure par rapport à la fréquence de Nyquist
    norm_cutoff = cutoff / nyquist

    # Conception du filtre passe-haut avec la méthode de Butterworth
    b, a = butter(order, norm_cutoff, btype='high', analog=False)

    # Application du filtre au signal audio
    return lfilter(b, a, data)


In [ ]:
def preprocess_audio(input_path, output_path):
    """
    Prétraite un fichier audio en appliquant :
    - un chargement à 16 kHz
    - un filtre passe-haut
    - une normalisation du signal
    - une suppression du silence au début/à la fin
    - un enregistrement du résultat prétraité

    Paramètres :
    - input_path : chemin du fichier audio d'entrée
    - output_path : chemin du fichier audio de sortie
    """

    # Chargement de l'audio avec une fréquence d'échantillonnage de 16 kHz
    y, sr = librosa.load(input_path, sr=16000)

    # Application d’un filtre passe-haut pour supprimer les basses fréquences (ex : bruit de fond)
    y = highpass_filter(y, sr)

    # Normalisation du signal pour que l'amplitude maximale soit 1 (entre -1 et 1)
    y = y / max(abs(y))

    # Suppression des silences au début et à la fin du signal
    y, _ = librosa.effects.trim(y, top_db=20)

    # Sauvegarde du signal prétraité dans le fichier de sortie
    sf.write(output_path, y, sr)

In [ ]:
import os

# Pour chaque chemin audio dans la colonne 'full_path' du DataFrame
for path in df['full_path']:
    # Récupère uniquement le nom du fichier (sans le dossier)
    filename = os.path.basename(path)

    # Construit le chemin de sortie pour le fichier audio prétraité
    output_file = f'/content/processed_audio/{filename}'

    # Crée le dossier de sortie s'il n'existe pas déjà
    os.makedirs('/content/processed_audio', exist_ok=True)

    # Applique le prétraitement audio et enregistre le fichier dans le dossier de sortie
    preprocess_audio(path, output_file)

# Ajoute une nouvelle colonne au DataFrame contenant les chemins des fichiers audio prétraités
df["processed_path"] = df["full_path"].apply(lambda x: f"/content/processed_audio/{os.path.basename(x)}")

<ipython-input-14-b972c3641d30>:4: RuntimeWarning: invalid value encountered in divide
  y = y / max(abs(y))


Etape 4 : Le test du modèle sur la dataset après l'application des techniques de preprocessing

In [ ]:
import pandas as pd
import whisper
from jiwer import wer

on utilise le modèle whisper

In [ ]:
model = whisper.load_model("medium")

In [ ]:
transcriptions = []
errors = []

In [ ]:
# Parcourt chaque ligne du DataFrame df avec son index
for idx, row in df.iterrows():
    try:
        # Transcrit l'audio prétraité à l'aide du modèle
        result = model.transcribe(row["processed_path"])

        # Extrait le texte prédit de la transcription
        predicted_text = result["text"]

        # Ajoute la transcription à la liste des transcriptions
        transcriptions.append(predicted_text)

        # Récupère le texte de référence (vérité terrain), converti en minuscules
        # Si la cellule est vide ou manquante, on met une chaîne vide
        ground_truth = str(row["text"]).lower() if pd.notna(row["text"]) else ""

        # Calcule l'erreur de reconnaissance (Word Error Rate) entre le texte réel et le texte prédit
        error = wer(ground_truth, predicted_text.lower())

        # Ajoute l'erreur à la liste des erreurs
        errors.append(error)

        # Affiche la prédiction pour le fichier audio courant
        print(f"{row['filename']}: {predicted_text}")

    except Exception as e:
        # En cas d’erreur pendant la transcription, affiche un message et ajoute des valeurs par défaut
        print(f"Failed on {row['filename']}: {e}")
        transcriptions.append("")  # Ajoute une transcription vide
        errors.append(1.0)         # Ajoute une erreur maximale (100%)

NameError: name 'df' is not defined

In [ ]:
df["predicted_text"] = transcriptions
df["wer"] = errors

In [ ]:
df.to_csv("whisper_results.csv", index=False)

In [ ]:
average_wer = sum(errors) / len(errors)
print(f"Average Word Error Rate: {average_wer:.4f}")